#Installation Process

# New Section

In [99]:
# install selenium and other resources for crawling data
!pip install selenium
# install other resources for doing crawling
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (89.0.4389.90-0ubuntu0.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [100]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
import json
import re

from selenium import webdriver
# Set driver for Chrome
options = webdriver.ChromeOptions()
options.add_argument('-headless') # since we run selenium on Google Colab so we don't want a chrome browser opens, so it will run in the background
options.add_argument('-no-sandbox')

In [101]:
def get_url(url):
  driver = webdriver.Chrome('chromedriver',options=options)        # Define the chrome drivers with setting options we define above    
  driver.get(url)                                             # Open the browser again to get web page
  html_data = driver.page_source                                   # After driver.get() is done, you can get back HTML string by using .page_source
  driver.close()                                                   # Close the driver after retrieving the web page

  soup = BeautifulSoup(html_data, 'html.parser') 

  return soup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [103]:
import sqlite3

In [104]:
PATH_TO_DB = '/content/drive/MyDrive/'

In [ ]:
conn = sqlite3.connect(PATH_TO_DB+'tiki.db')
cur = conn.cursor()

In [105]:
def select_query(query,conn=conn):
  return pd.read_sql_query(query,conn)

# Continuing with the set up provided. This next step is to do with setting up the Category Table. 

In [106]:
# Create table categories in the database using a function
def create_categories_table():
    query = """
        CREATE TABLE IF NOT EXISTS categories (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255),
            url TEXT, 
            parent_id INTEGER, 
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)
        
create_categories_table()

# The next step is to creating a class Category and the instance method is to save_into_db()

In [107]:
class Category:
    def __init__(self, name, url, parent_id=None, cat_id=None):
        self.cat_id = cat_id
        self.name = name
        self.url = url
        self.parent_id = parent_id

    def __repr__(self):
        return f"ID: {self.cat_id}, Name: {self.name}, URL: {self.url}, Parent: {self.parent_id}"

    def save_into_db(self):
        query = """
            INSERT INTO categories (name, url, parent_id)
            VALUES (?, ?, ?);
        """
        val = (self.name, self.url, self.parent_id)
        try:
            cur.execute(query, val)
            self.cat_id = cur.lastrowid
            conn.commit()
        except Exception as err:
            print('ERROR BY INSERT:', err)


# Get main categories

In [109]:
# empty our category set
CATEGORY_SET = set()

In [110]:
def can_add_to_cat_set(cat_name,save=False):
  if cat_name not in CATEGORY_SET:
    if save:
      CATEGORY_SET.add(cat_name)
      print(f'Added "{cat_name}" to CATEGORY_SET')
    return True
  return False

In [111]:
def get_main_category(main_categories,save_db=False):
    result=[]
    for i in main_categories:
        _=can_add_to_cat_set(i['Name'],save_db)

        main_cat = Category(i['Name'],i['URL'])
        if save_db:
            main_cat.save_into_db()
        result.append(main_cat)
    return result

In [ ]:
main_cat_objs = get_main_category(main_categories,save_db=True)

# The next step is to get the SUB CATEGORIES

In [112]:
import re

# get_sub_categories() given a parent category
def get_sub_categories(parent_category, save_db=False):
    parent_url = parent_category.url
    result = []

    try:
        soup = get_url(parent_url)
        for a in soup.find_all('a', {'class':'item item--category '}):
            name = a.text.strip()
            if can_add_to_cat_set(name,save_db): 
              sub_url = a['href']
              cat = Category(name, sub_url, parent_category.cat_id) # we now have parent_id, which is cat_id of parent category
              if save_db:
                  cat.save_into_db()
              result.append(cat)
    except Exception as err:
        print('ERROR IN GETTING SUB CATEGORIES:', err)
    return result

In [ ]:
main_cat_objs[0]

In [ ]:
select_query('SELECT * FROM categories')

,id,name,url,parent_id,create_at
0,1,Điện Thoại - Máy Tính Bảng,https://tiki.vn/dien-thoai-may-tinh-bang/c1789...,NaN,2021-04-09 15:11:02
1,2,Điện Tử - Điện Lạnh,https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?...,NaN,2021-04-09 15:11:02
2,3,Phụ Kiện - Thiết Bị Số,https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815...,NaN,2021-04-09 15:11:02
3,4,Laptop - Thiết bị IT,https://tiki.vn/laptop-may-vi-tinh/c1846?src=c...,NaN,2021-04-09 15:11:02
4,5,Máy Ảnh - Quay Phim,https://tiki.vn/may-anh/c1801?src=c.1801.hambu...,NaN,2021-04-09 15:11:02
...,...,...,...,...,...
2691,2692,Mũ Fullface,https://tiki.vn/mu-fullface/c46186?src=c.8594....,2150.0,2021-04-09 21:38:33
2692,2693,Mũ lật cằm,https://tiki.vn/mu-lat-cam/c46192?src=c.8594.h...,2150.0,2021-04-09 21:38:33
2693,2694,Bảo dưỡng - Đánh bóng,https://tiki.vn/bao-duong-danh-bong/c45518?src...,2161.0,2021-04-09 21:40:23
2694,2695,"Dung môi, dung dịch",https://tiki.vn/dung-moi-dung-dich/c45514?src=...,2161.0,2021-04-09 21:40:23


# The next question is what if the categories dont have any sub-categories?

# Below we try to solve this problem.

In [113]:
def get_all_categories(categories,save_db):
    categories = categories.copy()
    while len(categories):
        cat_to_crawl = categories[0]
        print(f'Getting {cat_to_crawl} sub-categories...')
        sub_categories = get_sub_categories(cat_to_crawl, save_db=save_db)
        print(f'Finished! {cat_to_crawl.name} has {len(sub_categories)} sub-categories')
        categories+=sub_categories
        del categories[0]

In [ ]:

create_categories_table()

In [ ]:
# empty our category set
CATEGORY_SET= set()

In [ ]:
main_cat_objs = get_main_category(main_categories,save_db=True)

In [ ]:
main_cat_objs

In [ ]:
get_all_categories(main_cat_objs,save_db=True)

In [ ]:
select_query('SELECT COUNT(*) FROM categories')

,COUNT(*)
0,2697


In [ ]:
select_query('select * from categories')

,id,name,url,parent_id,create_at
0,1,Điện Thoại - Máy Tính Bảng,https://tiki.vn/dien-thoai-may-tinh-bang/c1789...,NaN,2021-04-09 15:11:02
1,2,Điện Tử - Điện Lạnh,https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?...,NaN,2021-04-09 15:11:02
2,3,Phụ Kiện - Thiết Bị Số,https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815...,NaN,2021-04-09 15:11:02
3,4,Laptop - Thiết bị IT,https://tiki.vn/laptop-may-vi-tinh/c1846?src=c...,NaN,2021-04-09 15:11:02
4,5,Máy Ảnh - Quay Phim,https://tiki.vn/may-anh/c1801?src=c.1801.hambu...,NaN,2021-04-09 15:11:02
...,...,...,...,...,...
2691,2692,Mũ Fullface,https://tiki.vn/mu-fullface/c46186?src=c.8594....,2150.0,2021-04-09 21:38:33
2692,2693,Mũ lật cằm,https://tiki.vn/mu-lat-cam/c46192?src=c.8594.h...,2150.0,2021-04-09 21:38:33
2693,2694,Bảo dưỡng - Đánh bóng,https://tiki.vn/bao-duong-danh-bong/c45518?src...,2161.0,2021-04-09 21:40:23
2694,2695,"Dung môi, dung dịch",https://tiki.vn/dung-moi-dung-dich/c45514?src=...,2161.0,2021-04-09 21:40:23


# Taking that dataFrame and eliminating duplicates, Finding "lowest" sub category + finding urls

In [ ]:
def find_main_cat(parent_id):
    p_id = parent_id
    if p_id <= 15:
        return p_id

    query = f'''
        SELECT parent_id
        FROM categories
            WHERE id = {p_id}
    '''

    p_id = pd.read_sql_query(query, conn).iloc[0,0]
    return find_main_cat(p_id)

In [114]:
find_main_cat(2159)

12

In [115]:


query = '''
SELECT c1.id, c1.name,c1.url,c1.parent_id 
FROM categories as c1

EXCEPT

SELECT c1.id, c1.name,c1.url,c1.parent_id 
FROM categories as c1
JOIN categories as c2
ON c1.id = c2.parent_id
GROUP BY c1.id
'''
sub_cat = pd.read_sql_query(query, conn)
sub_cat


,id,name,url,parent_id
0,16,Điện thoại Smartphone,https://tiki.vn/dien-thoai-smartphone/c1795?sr...,1.0
1,17,Điện thoại phổ thông,https://tiki.vn/dien-thoai-pho-thong/c1796?src...,1.0
2,18,Điện thoại bàn,https://tiki.vn/dien-thoai-ban/c8061?src=c.178...,1.0
3,19,Máy đọc sách,https://tiki.vn/may-doc-sach/c28856?src=c.1789...,1.0
4,20,Máy tính bảng,https://tiki.vn/may-tinh-bang/c1794?src=c.1789...,1.0
...,...,...,...,...
2234,2695,"Dung môi, dung dịch",https://tiki.vn/dung-moi-dung-dich/c45514?src=...,2161.0
2235,2696,Phone-Tablet,https://tiki.vn/dien-thoai-may-tinh-bang/c1789,NaN
2236,2697,Phone-Tablet,https://tiki.vn/dien-thoai-may-tinh-bang/c1789,NaN
2237,2698,Phone-Tablet,https://tiki.vn/dien-thoai-may-tinh-bang/c1789,NaN


In [116]:
sub_cat.to_csv("./lower_sub_cat.csv", index = False)


In [117]:
query1 = '''

SELECT *
FROM categories
WHERE parent_id in (1,2,3)
  
    
'''
a= pd.read_sql_query(query1, conn)


a

,id,name,url,parent_id,create_at
0,16,Điện thoại Smartphone,https://tiki.vn/dien-thoai-smartphone/c1795?sr...,1,2021-04-09 15:11:32
1,17,Điện thoại phổ thông,https://tiki.vn/dien-thoai-pho-thong/c1796?src...,1,2021-04-09 15:11:32
2,18,Điện thoại bàn,https://tiki.vn/dien-thoai-ban/c8061?src=c.178...,1,2021-04-09 15:11:32
3,19,Máy đọc sách,https://tiki.vn/may-doc-sach/c28856?src=c.1789...,1,2021-04-09 15:11:32
4,20,Máy tính bảng,https://tiki.vn/may-tinh-bang/c1794?src=c.1789...,1,2021-04-09 15:11:32
5,21,Âm thanh & Phụ kiện Tivi,https://tiki.vn/am-thanh-phu-kien-tivi/c26568?...,2,2021-04-09 15:11:45
6,22,Tủ lạnh,https://tiki.vn/tu-lanh/c2328?src=c.4221.hambu...,2,2021-04-09 15:11:45
7,23,Máy lạnh - Máy điều hòa,https://tiki.vn/may-lanh-may-dieu-hoa/c3865?sr...,2,2021-04-09 15:11:45
8,24,Máy giặt,https://tiki.vn/may-giat/c3862?src=c.4221.hamb...,2,2021-04-09 15:11:45
9,25,Tủ đông - Tủ mát,https://tiki.vn/tu-dong-tu-mat/c3868?src=c.422...,2,2021-04-09 15:11:45


# Before Creating the Product Class OOP we need to create a PRODUCT TABLE

In [136]:
def create_products_table():
    """ CREATE TABLE FOR PRODUCTS"""
    query = """
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255),
            price VARCHAR (255), 
            product_url VARCHAR(255), 
            image VARCHAR(255), 
            product_sku VARCHAR(255), 
            tiki_now TEXT, 
            freeship TEXT, 
            review VARCHAR(255), 
            rating FLOAT, 
            under_price TEXT, 
            discount TEXT, 
            installment TEXT, 
            gift TEXT, 
            cat_id INTEGER
          
        )
    """
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)

create_products_table()

In [134]:
# cur.execute('DROP TABLE products;')
# conn.commit()

In [137]:
select_query('SELECT * FROM products')

,id,name,price,product_url,image,product_sku,tiki_now,freeship,review,rating,under_price,discount,installment,gift,cat_id


# Creating product table an OOP

In [124]:
class Product:
    def __init__(self, name, price, product_url, image, product_sku, tiki_now, freeship, review, rating, under_price, discount, installment, gift, cat_id):
        self.name = name
        self.price = price
        self.product_url = product_url
        self.image = image
        self.product_sku = product_sku
        self.tiki_now = tiki_now
        self.freeship = freeship
        self.review = review
        self.rating =rating
        self.under_price =under_price
        self.discount = discount
        self.installment = installment
        self.gift = gift
        self.cat_id = cat_id

      
    def __repr__(self):
        return f" Product_name:{self.name}, Price:{self.price}, Product_URL: {self.product_url}, Image_URL: {self.image}, SKU: {self.product_sku}, TikiNow: {self.tiki_now}, Freeship: {self.freeship}, Review: {self.review}, Rating: {self.rating}, Refund: {self.under_price}, Discount: {self.discount}, Installment: {self.installment}, Gift: {self.gift}, Category_id: {self.cat_id}"

    def save_into_db(self):
        query1 = """
            INSERT INTO products (name, price, product_url, image, product_sku, tiki_now, freeship, review, rating, under_price, discount, installment, gift, cat_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        """
        val1 = (self.name, self.price, self.product_url, self.image, self.product_sku, self.tiki_now, self.freeship, self.review, self.rating, self.under_price, self.discount, self.installment, self.gift, self.cat_id)
        try:
            cur.execute(query1, val1)
            self.id = cur.lastrowid
            conn.commit()
        except Exception as err:
            print('ERROR BY INSERT:', err)


# Extract info from all products of a specfic category on Tiki website

In [126]:
def extract_tiki_info(cat,url,save_db=True):
  """ Extract info from all products of a specfic category on Tiki website
      Input: url
      Output: info of products, saved as list of dictionary. If no products shown, return empty list.
  """
  data = []
  index = 1
  soup = get_url(url)

  # FIND ALL PRODUCT ITEMS
  products = soup.find_all('a', {'class':'product-item'})
  all_script = soup.find_all('script', {'type':'application/ld+json'})
  print("BATCH SIZE:", len(products))

  if (soup.find('div', {'class':'style__StyledNotFoundProductView-sc-1uz0b49-0'})):
        print ("END")
 
  elif len(products):
        # EXTRACT INFO TO DICTIONARY
    for i in products: 
        d = {'name':'','price':'','product_url':'','image':'', 'product_sku':'',
            'tiki_now':'','freeship':'','review':'','rating':'','under_price':'',
            'discount':'','installment':'','gift':'', 'cat_id': ''}
        
        try:
            d['name']         = i.find('div',{'class' : 'name'}).text
            d['price']        = int(re.sub('[. ₫]','', i.find('div',{'class':'price-discount__price'}).text))
            d['product_url']  = 'https://tiki.vn' + i['href'] 
            thumbnail         = i.find('div',{'class':'thumbnail'})
            d['image']        = thumbnail.img['src']        
            d['tiki_now']     = bool(i.find('div',{'class':'badge-service'}).find('div',{'class':'item'})) 
            d['freeship']     = bool(i.find('div',{'class':'badge-top'}).text == "Freeship")
            
            if i.find('div',{'class':'review'}):
                d['review']     = int(i.find('div',{'class':'review'}).text.strip('(').strip(')'))
            else:
                d['review'] = "N/A"
            
            d['under_price']  = bool(i.find('div',{'class':'badge-under-price'}).find('div',{'class':'item'}))

            if i.find('div', {'class':'price-discount__discount'}):
                d['discount']   = int(re.sub('[-%]','', i.find('div',{'class':'price-discount__discount'}).text))
            else:
                d['discount']   = "N/A"
            
            d['installment']  = bool(i.find('div',{'class':'badge-benefits'}).img)
            d['gift']         = bool(i.find('div',{'class':'freegift-list'}))

            script = all_script[index]
            dict_content = json.loads(script.text)
            d['product_sku']  = dict_content['sku']
            
            if 'aggregateRating' in dict_content:
                d['rating']     = float(dict_content['aggregateRating']['ratingValue'])
            else:
                d['rating']     = "N/A"
            d['cat_id'] = cat
        except Exception as e:
            print(e)

        index += 1
        prod = Product(d['name'],d['price'], d['product_url'], d['image'], d['product_sku'], d['tiki_now'], d['freeship'], d['review'], d['rating'], d['under_price'], d['discount'], d['installment'], d['gift'], d['cat_id'])
        if save_db:
                prod.save_into_db()
        
        data.append(d)
        
        
  return data



In [131]:
def scrape_tiki(cat,base_url):
  result = []
  page_number = 1
  main, opt = base_url.split('?')
  
  while page_number <= 2:
    page = f'?page={page_number}&'
    url = main+page+opt
    print("url =", url)
    data = extract_tiki_info(cat,url)
    if len(data) > 0:
      result.extend(data)
    else:
      break
    page_number += 1

    sleep(randint(1,2))

  print("****TOTAL = ",len(result))

  items = pd.DataFrame(data = result, columns = result[0].keys())
  items.to_csv("./result2.csv", index=False)


SAVE JOINED TABLE TO CSV TYPE AND GET THE LINKS FROM THE FILE FOR SCRAPING

In [ ]:


for i in range(len(data_subcat)):
    url = data_subcat['url'][i]
    cat = int(data_subcat['id'][i])
    scrape_tiki(cat,url)

url = https://tiki.vn/dien-thoai-smartphone/c1795?page=1&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 48
url = https://tiki.vn/dien-thoai-smartphone/c1795?page=2&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 48
****TOTAL =  96
url = https://tiki.vn/dien-thoai-pho-thong/c1796?page=1&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 48
url = https://tiki.vn/dien-thoai-pho-thong/c1796?page=2&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 48
****TOTAL =  96
url = https://tiki.vn/dien-thoai-ban/c8061?page=1&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 48
url = https://tiki.vn/dien-thoai-ban/c8061?page=2&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 46
****TOTAL =  94
url = https://tiki.vn/may-doc-sach/c28856?page=1&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 41
url = https://tiki.vn/may-doc-sach/c28856?page=2&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 0
END
****TOTAL =  41
url = https://tiki.vn/may-tinh-bang/c1794?page=1&src=c.1789.h

In [4]:
query = '''

SELECT *
FROM products

  
    
'''
pd.read_sql_query(query, conn)

NameError: ignored